In [29]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from pathlib import Path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

<IPython.core.display.Javascript object>

In [ ]:
# To notify when cell is complete (may need to delete first line)
!pip uninstall jupyternotify -y
!pip install git+https://github.com/cphyc/jupyter-notify.git
%reload_ext jupyternotify

In [ ]:
# read in cleaned CSV
filepath = Path('../massive.csv')
massive = pd.read_csv(filepath)
massive = massive.sample(n=15000)

In [23]:
massive.head(1)

,id,title,audienceScore,tomatoMeter,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,delta
49099,john_wick_chapter_2,John Wick: Chapter 2,85.0,89.0,2377946,2017-02-09,Jim Vorel,False,fresh,Paste Magazine,Holding the torch passed from '80s and '90s Jo...,POSITIVE,4.0


<IPython.core.display.Javascript object>

In [ ]:
# Dropping columns including audienceScore because it could unfairly help the model predict the delta. Leaving tomatoMeter to give the model a base of where the delta could be.
massive = massive.drop(columns=['id', 'reviewId', 'creationDate', 'isTopCritic', 'reviewState', 'audienceScore'])

In [27]:
massive.isnull().sum()

title             0
tomatoMeter       0
criticName        0
publicatioName    0
reviewText        0
scoreSentiment    0
delta             0
dtype: int64

In [28]:
# Dropping duplicates
massive = massive.drop_duplicates(subset='reviewText', keep='first')
# Checking for duplicates
dup_df = massive.apply(lambda x: x.duplicated()).sum()
dup_df

title              7256
tomatoMeter       14891
criticName        11847
publicatioName    13465
reviewText            0
scoreSentiment    14990
delta             14871
dtype: int64

In [227]:
# define a function to process text on the data
lemmatizer = WordNetLemmatizer()
import re
def process_text(text): 
    sw = set(stopwords.words('english')) 
    regex = re.compile("[^a-zA-Z ]") 
    re_clean = regex.sub('', text) 
    words = word_tokenize(re_clean) 
    lem = [lemmatizer.lemmatize(word) for word in words] 
    output = ' '.join([word.lower() for word in lem if word.lower() not in sw]) 
    return output

In [228]:
# use a lambda x function to apple process text on the whole column.
massive['reviewText'] = massive['reviewText'].apply(lambda x: process_text(x))

In [229]:
# tfidf vectorizer for sentiment model
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(massive['reviewText'])
y = massive['scoreSentiment']

In [230]:
# Split the data into training and testing sets for sentiment model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Choose a machine learning model (e.g., Logistic Regression) and train it
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [231]:
model.score(X_train, y_train)

0.8227442605445809

In [232]:
model.score(X_test, y_test)

0.7589428723972237

In [233]:
# Creating a dataframe made of the vectorized review text for the linear model
massive_features = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())

C:\Users\Dan\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [234]:
massive_features.head(2)

,abandon,ability,able,abrams,absence,absolute,absolutely,absorbing,absurd,absurdist,...,youth,youthful,youve,zany,zealand,zemeckis,zero,zippy,zombie,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.265617,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [235]:
# Need to rename column because title is also a column in massive_features
massive = massive.rename(columns={'title': 'title_'})

In [ ]:
# Vectorizing original df to dense array for linear model
tfidf_dense = tfidf_vectorizer.fit_transform(massive['reviewText']).todense()

# Naming columns for each vectorized word and combining with original massive dataframe
new_cols = tfidf_vectorizer.get_feature_names()
combined = massive.join(pd.DataFrame(tfidf_dense, columns=new_cols))

C:\Users\Dan\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [245]:
# Dropping review text now that vectorized words are all columns
combined = combined.drop(columns=['reviewText'])

In [238]:
# Fill the nulls in all of the word columns with 0
combined = combined.fillna(0)

In [ ]:
combined.isnull().sum().head(9)

title_            0
tomatoMeter       0
criticName        0
publicatioName    0
scoreSentiment    0
delta             0
abandon           0
ability           0
able              0
dtype: int64

In [ ]:
# Renaming the least-appearing critics as Other to be encoded as the same critic later
counts = combined.criticName.value_counts()
threshold = combined.criticName.isin(counts.index[counts<16])
combined.loc[threshold, 'criticName'] = 'Other'
combined['criticName'].value_counts()

Other              8954
Dennis Schwartz     111
David Nusair         94
Roger Ebert          89
Roger Moore          81
                   ... 
Ella Taylor          16
Scott Foundas        16
Dann Gire            16
David Noh            16
Kim Newman           16
Name: criticName, Length: 214, dtype: int64

In [ ]:
# Repeat for publications
counts = combined.publicatioName.value_counts()
threshold = combined.publicatioName.isin(counts.index[counts<12])
combined.loc[threshold, 'publicatioName'] = 'Other'
combined['publicatioName'].value_counts()

Other                 4148
New York Times         180
Variety                166
Los Angeles Times      138
Slant Magazine         134
                      ... 
The Ringer              12
Three Movie Buffs       12
Film Companion          12
Baltimore Sun           12
culturevulture.net      12
Name: publicatioName, Length: 333, dtype: int64

In [271]:
combined.head(1)

,tomatoMeter,delta,abandon,ability,able,abrams,absence,absolute,absolutely,absorbing,...,publicatioName_culturevulture.net,publicatioName_eFilmCritic.com,publicatioName_eye WEEKLY,publicatioName_film-authority.com,publicatioName_indieWire,publicatioName_jackiekcooper.com,publicatioName_rachelsreviews.net,publicatioName_rec.arts.movies.reviews,scoreSentiment_NEGATIVE,scoreSentiment_POSITIVE
328556,22.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# list of columns to dummy
categorical_cols = ['title_', 'criticName', 'publicatioName', 'scoreSentiment'] 
# get dummies on categorical columns
combined = pd.get_dummies(combined, columns = categorical_cols)

In [273]:
X = combined.drop(columns=['delta'])
y = combined['delta']

In [274]:
X.isnull().sum().sum()

0

In [284]:
y[:5]

328556   -10.0
477873    19.0
709010     9.0
340594     8.0
720402   -31.0
Name: delta, dtype: float64

In [292]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [293]:
X_train.isnull().sum().sum()

0

In [283]:
# Scaling X
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [339]:
pca = PCA(n_components=2000)
X_train_p = pca.fit_transform(X_train_sc)
X_test_p = pca.transform(X_test_sc)

In [340]:
# Choose a machine learning model (e.g., Logistic Regression) and train it
model = LinearRegression()
model.fit(X_train_p, y_train)

LinearRegression()

In [341]:
model.score(X_train_p, y_train)

0.5509262446941581

In [342]:
model.score(X_test_p, y_test)

0.3442473356373843